## Introduction 

Here we are trying to adjust parameters of a paraphrase model to generate adversarial examples. 
### Policy gradients 
The key parameter update equation is $\theta_{t+1} = \theta_t + \alpha \nabla_\theta J(\theta)$, where $\alpha$ is a step size parameter, the parameter vector $\theta$ is for a model (here a paraphrase model), and $J$ is a loss function. The time step $t$ depends on the problem specification and we will get to it later. 

Now in my review I have defined the loss function $J(\theta) = E_\pi[r(\tau)]$. Here: 
* $\pi$ is the policy, a probability distribution for the next action in a given state; essentially $p(a_t|s_t)$
* $\tau$ is a trajectory, a specific sequence $s_0, a_0, r_1, s_1, a_1, \ldots$ of the agent in the game. This starts at time $t=0$ and finishes at time $t=T$. 
* $r(\tau)$ is the sum of rewards for a trajectory $\tau$, or in other words, the total reward for the trajectory. 

For this loss function higher values are better (which might make it a reward function) and so we might have to invert it at some point. 

To update parameters we must find the gradient $\nabla_\theta J(\theta)$, which measures how $J(\theta)$ changes when we adjust the parameters of the paraphrase model. The gradient is simplified through some maths to get the policy gradient theorem $$ \nabla_\theta J(\theta) =  \nabla_\theta E_\pi [r(\tau)]  = E_\pi \left[r(\tau) \sum_{t=1}^T \nabla_\theta \log \pi (a_t|s_t)  \right] $$ 

To calculate this you need to calculate the expectation term, which in turn means evaluating every possible trajectory $\tau$ and its expected return. Generally this is not possible and instead we turn to estimators.  

One of these is REINFORCE. It gives us  $$ \nabla_\theta J(\theta) \approx \sum_{s=1}^S \sum_{t=1}^T G_t \nabla \log \pi(a_t|s_t)$$ where 
* $G_t$ is the discounted return and is given by $G_t = r_t + \beta r_{t-1} + \beta^2 r_{t-2} + \dots$. It's a rough estimate of $r(\tau)$. Rewards obtained later in the episode are weighted much higher than rewards obtained earlier. I guess it assumes that the parameters update every timestep. 
* $S$ is some number of samples.

The implementation of REINFORCE and similar estimators depends on how we formulate the problem. Below we present some possible formulations

### Interpretation One: Document-level  
This is the first implementation we will try. 

Here we generate a list of paraphrases at each time point. The idea is that there is one paraphrase amongst them that is a good adversarial example. We try to tune the model to produce the best one. 

This interpretation sees forming the complete paraphrase as one time step. So it isn't token-level but document-level. 

* Starting state: $s0 = x$, the original example  
* Actions: each action is "choosing" a paraphrase (or of choosing $n$ paraphrases). The set of all possible paraphrases and their probabilities is the policy. So $\pi(a|s) = p(x'| x;\theta)$ where $x'$ is the paraphrase (or list of paraphrases). 
    * To approximate this probability, what we can do is generate a large list of paraphrases, and for each, the probabilities of generating each token in turn for that paraphrase. This gives a rough "probability" of how likely that sequence was. This number is kind of like a weight for how good that paraphrase is, according to the model.  We can then turn the weights into probabilities to get a "probability" of the paraphrase. This is dependent on the number of paraphrases generated, so generating a large list is likely to be better for this task. 
* Reward: The paraphrase moves through the reward function $R(x, x')$) to get the reward $r$. 
* Time steps: We only have one time step in the game ($T=1$ and $G_t=r$)  


There are a few variations to this scenario that we can do. For each of these we will formulate the policy and the reward function $R$. Below, $x'$ means paraphrase, $f(x)_y$ means the model confidence of x for the class of the true label $y$, $SS(a,b)$ is the result of a semantic similarity model run over $a$ and $b$, and $\lambda$ is a hyperparameter.  


#### One-paraphrase 
Here we only generate one paraphrase. This scenario also has a few options. First we generate a list of paraphrases with the probabilities of selecting one. Then we either sample probabilistically from the list or pick the most probable option. 

In this case the policy $p(x'|x,\theta)$ is the chance of obtaining a specific paraphrase. For the sampling option this is equal to its sample probability. For the top option this is just the probability of selecting that option. 

The reward function might look like $R(x,x') = f(x)_y - f(x')_y + \lambda SS(x, x')$. We could also make the $SS$ factor a step-function above some threshold. 

The REINFORCE equation $$ \nabla_\theta J(\theta) \approx \sum_{s=1}^S \sum_{t=1}^T G_t \nabla \log \pi(a_t|s_t)$$ becomes $$ \nabla_\theta J(\theta) \approx \sum_{s=1}^S  R(x,x'_s) \nabla \log p(x'_s|x,\theta)$$ We repeat the process $S$ times where $S$ is ideally as large as possible. We can start with something simple (e.g. $S=10$ or $S=100$) and go from there.  

The gradient term $\nabla \log p(x'_s|x,\theta)$ can hopefully be found with autodiff. 

#### Set of paraphrases
In this scenario the paraphrase model is evaluated on performance over a set of paraphrases, which we call $X'$ here. The policy becomes $p(X'|x, \theta)$, the probability of obtaining that list. We can get this probability by multipling together the "probability" of each individual paraphrase, multiplying also by nCr (for r paraphrases out of n total) to account for the lack of order in the list. 

We can make a number of sub-scenarios here. 

For the **top-paraphrase in set** condition the paraphrase generator is only measured on the best reward for a paraphrase in its set. The idea is the generator will learn to produce a diverse set of examples, any of which could plausibly be a good adversarial example. Here we only look at best performing paraphrase $x'_m$, which we can find by $x'_m = \max_i [f(x)_y - f(x'_i)_y]$, then return $R(x,x'_m) = [f(x)_y - f(x'_m)_y] + \lambda SS(x,x'_m)$ 

For the **average-paraphrase in set** condition the paraphrase generator is measured on the average reward of the paraphrases in its set. This encourages the generator to consider performance of all examples more-or-less equally. The reward function could be something like $\frac{1}{k} \sum_{i=1}^k \left[ f(x)_y - f(x'_i)_y + \lambda SS(x, x'_i) \right]$ 

A combination of these scenarios is the **top-k/top-p\% paraphrases in set**. Here we only use the top-$k$ paraphrases, or more generally, the top $p$ percentage of paraphrases. 


### Interpretation 2: Token-level
This interpretation is at token-level; it sees choosing the next word as the next time step. 

* Starting state: $s0 = x$, the initial state. But you also have a "blank slate" for the paraphrase. So maybe it's a tuple (x, pp) where pp is a paraphrase with no words. Here x is used as the reference for the paraphrase generator.  
* Actions: Choose the next word of p. I guess this starts with the \<START\> token (or something similar). Then you have the policy $\pi(a|s)$ which is the same as $p(w_{next}|pp, x; \theta)$ where $\theta$ is the paraphrase model parameters, $pp$ is the so-far constructed sentence, and $w_{next}$ is the next token (I say token because I don't know if this model is on the subword or word basis). 
* Time steps: every token is generated one-by-one and each of these is allocated a time step. This means probably that you also update the parameters after each token generated too. 
* Reward. The reward is allocated every token. There are many reward functions (see papers on token-level loss functions). Some also incorporate document-level rewards too. 
* Next state. $s_1$ is again the tuple $(x, pp)$ but now $pp$ has the first word in it. 

On *teacher forcing*. This is when you have a ground-truth paraphrase and you can use it when generating tokens. This is useful because if the model makes a mistake it doesn't continue down that track but is adjusted back. This stops big divergences (but also might limit the diversity of generated paraphrases). This is used when training a paraphrase model. You have a set of reference paraphrases that are human provided. Here though we only have the original sentence and no references. We could generate adversarial examples and use that to do teacher forcing. Generating them using textattack recipes might work. This is only really used on the token-level rewards. 

### Updating the paraphrase model parameters. 

There is a choice here. We can either directly update the parameters of the paraphrase model. Or we can fix the parameters and add a new dense layer to the end of the model. We could then train this dense layer to convert paraphrases to adversarial paraphrases. 

Before trying this out, I am worried that we will destroy the capabilities of the paraphrase generator a bit. We might get semantically invalid or ungrammatical or gibberish text. If so we could try and mitigate it a bit by shaping our reward function to maintain grammatical components. 

### Experiment order

Plan is to try the following order: 

1. One-paraphrase (most probable option). I'll start with this one because it is probably the most simple case. Within this category: 
    1a. tune existing parameters only (see if the text is recognisable) 
    1b. add dense layer onto end and try again 
2. One-paraphrase (sampled). This seems like a logical extension on the first one. 
3. Paraphrase-set options. (Decide after finishing 1, 2) 
4. Token-level tuning. (Decide after 1,2,3)


### Layer Freezing

I am uncertain on if to do this or not. 

* This [paper](https://arxiv.org/abs/1911.03090) indicates that you can get pretty good results by freezing all layers except the last few 
* Conversely I saw in the transformers documentation that transformers train better if you don't do layer freezing 


## Setup, load models + datasets 

In [1]:
%load_ext autoreload
%autoreload 2
%load_ext line_profiler

In [2]:
## Imports and environment variables 
import os
os.environ["TOKENIZERS_PARALLELISM"]  = "true"  # set to false if not working

# Core imports 
import torch, numpy as np, pandas as pd, gc,sys, logging, warnings
from torch.utils.data import DataLoader, RandomSampler
from datasets import load_dataset, load_metric, load_from_disk, DatasetDict
from transformers import (AutoModelForSeq2SeqLM, AutoModelForSequenceClassification, 
                          AutoTokenizer, AdamW, SchedulerType, get_scheduler)
from torch.distributions import Categorical
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import pytorch_cos_sim
from collections import defaultdict
from accelerate import Accelerator, notebook_launcher
from cachetools import cached, LRUCache
from types import MethodType
from timeit import default_timer as timer
import utils; from utils import *   # local script
from tests import *   # local script
from tqdm.auto import tqdm
import itertools
import copy 
import wandb
from undecorated import undecorated


# Dev imports (not needed for final script)
import seaborn as sns
from IPython.display import Markdown
from pprint import pprint
from IPython.core.debugger import set_trace
from GPUtil import showUtilization
import torchsnooper

In [3]:
#### Parameters, notes and training settings

### These parameters mostly don't do anything but are more notes (for the wanb.init function)
debug_run = "true"   # doesn't do anything
sampling_strategy = "greedy"  # doesn't do anything
# copy-paste this from reward function
reward_strategy = "[-0.5 if sts < 0.5 else 0.5+v*sts for v,sts in zip(vm_scores, sts_scores)]" # doesn't do anything
# options for the pp_model 
# 1. tuner007/pegasus_paraphrase
# 2. tdopierre/ProtAugment-ParaphraseGenerator
# 3. eugenesiow/bart-paraphrase
#pp_name = "eugenesiow/bart-paraphrase"
pp_name = "eugenesiow/bart-paraphrase"
vm_name = "textattack/distilbert-base-uncased-rotten-tomatoes"
sts_name = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
dataset_name = "simple"
n_layers_frozen = "2"  # counting from the back (doesn't do anything yet)
use_fp16 = True
save_model_while_training = False
save_model_freq = 10

# These parameters only have effect if small_ds = True

### Paraphrase parameters  
pp_model_params = {
    "num_beams": 1, 
    "num_return_sequences": 1, 
    "num_beam_groups": 1, 
    "diversity_penalty": 0.,   # must be a float
    "temperature": 1.5,
    "length_penalty" : 1,
    "min_length" : 5
}
if dataset_name == "rotten_tomatoes":
    max_length = 64
    pp_model_params['max_length'] = max_length
    #batch_size_train = 32
    #batch_size_eval = 128
    batch_size_train = 16
    batch_size_eval = 32
    accumulation_steps = 1
    n_train_epochs = 2
    eval_freq = 1 
    use_small_ds = True  # for testing purposes
    n_shards         = 60    if use_small_ds else None 
    shard_contiguous = False if use_small_ds else None 
elif dataset_name == "simple":
    max_length = 20
    pp_model_params['max_length'] = max_length
    batch_size_train = 2
    batch_size_eval = 4
    accumulation_steps = 1
    eval_freq = 10 
    n_train_epochs = 60
    use_small_ds = False  # for testing purposes
    n_shards         =  None 
    shard_contiguous =  None 
    if use_small_ds == True: 
        raise Exception("Don't shard when using the simple dataset (no need)")


### Training parameters
seed = 420
lr = 3e-5 # Initial learning rate (after the potential warmup period) to use
normalise_rewards = False
metrics = ['loss', 'pp_logp', 'reward', 'vm_score', "sts_score", 'label_flip']
pin_memory = True
zero_grad_with_none = False
pad_token_embeddings = True
padding_multiple = 8 
bucket_by_length = True
shuffle_train = False
remove_misclassified_examples = True
#weight_decay = 0
#lr_scheduler_type = 'none'
#n_warmup_steps = 30 

### W&B parameters
wandb_mode = "online"  # set to "disabled" to turn off wandb, "online" to enable it
wandb_log_grads = False   
wandb_log_grads_freq = 1  # no effect if wandb_log_grads is False
wandb_plot_examples = False
wandb_n_examples_plot = 4  # number of individual examples to plot curves for
# log a table to wandb with the examples and rewards the model sees while training. Useful for debugging 
# and seeing what is going on, but slows down training time. 
wandb_log_training_step_table = True  
wandb_log_token_entropy=True
wandb_log_token_probabilities = True

# Parameter dict
config_d = dict(
    debug_run = debug_run,
    sampling_strategy = sampling_strategy,
    reward_strategy = reward_strategy,
    pp_name = pp_name,
    vm_name = vm_name,
    sts_name=sts_name,
    dataset_name = dataset_name, 
    use_small_ds = use_small_ds,
    shard_params = dict(
        n_shards = n_shards,
        shard_contiguous = shard_contiguous,
    ),
    n_layers_frozen = n_layers_frozen,
    pp_model_params = pp_model_params, 
    seed = seed,
    batch_size_train = batch_size_train,
    batch_size_eval = batch_size_eval,
    fp16 = use_fp16,
    max_length = max_length,
    lr = lr, 
    accumulation_steps=accumulation_steps,
    n_train_epochs = n_train_epochs,
    eval_freq = eval_freq,
    normalise_rewards = normalise_rewards,
    metrics = metrics,
    pin_memory = pin_memory,
    zero_grad_with_none = zero_grad_with_none,
    pad_token_embeddings = pad_token_embeddings,
    padding_multiple = padding_multiple,
    bucket_by_length = bucket_by_length,
    shuffle_train = shuffle_train,
    remove_misclassified_examples = remove_misclassified_examples, 
    wandb_params=dict(
        log_grads = wandb_log_grads,
        log_grads_freq = wandb_log_grads_freq, 
        plot_examples = wandb_plot_examples, 
        n_examples_plot = wandb_n_examples_plot, 
        log_training_step_table = wandb_log_training_step_table
    )
)

In [4]:
#### Misc setup
# Paths
path_cache = './cache/'
path_results = "./results/"
path_data = "./data/"
path_checkpoints = "../model_checkpoints/travis_attack/"

# Seeds
torch.manual_seed(seed)
np.random.seed(seed)
torch.cuda.manual_seed(seed)

# Devices and GPU settings
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') 
#device = accelerator.device
devicenum = torch.cuda.current_device() if device.type == 'cuda' else -1
# When not using Accelerator
#n_wkrs = 4 * torch.cuda.device_count()
# When using Accelerator 
n_wkrs = 0 

# Configs
pd.set_option("display.max_colwidth", 400)
wandb.Table.MAX_ARTIFACT_ROWS = 1000000
wandb.Table.MAX_ROWS = 1000000
run_notes = f"Debug run:{debug_run}\nReward: {reward_strategy}\nDataset: {dataset_name}\
\nSampling strategy: {sampling_strategy}"

# Logging 
logging.basicConfig(format='%(message)s', stream=sys.stdout) # stdout while we are doing stdout to file piping
logger = logging.getLogger("main_logger")
logger.setLevel(logging.INFO)

# Other 
splits = ['train', 'valid', 'test']

### Dev code
# used to know what to set max_pp_length to. 
track_pp_sizes = False
if track_pp_sizes: 
    n_train_epochs = 1
    use_small_ds = True 
    n_shards = 2  # half the train dataset gives us a good understanding of this
    orig_max_l = []
    pp_max_l = []
    # After running the code  
    # sns.distplot(orig_max_l)
    # sns.distplot(pp_max_l)

### Load models

In [5]:
## Paraphrase (pp) model 
pp_tokenizer = AutoTokenizer.from_pretrained(pp_name)
# takes about 3GB memory space up on the GPU
# change the `local_files_only` argument if changing the model name 
pp_model = AutoModelForSeq2SeqLM.from_pretrained(pp_name, local_files_only=True)
# The no_grad version of generate
generate_with_grad = undecorated(pp_model.generate)
pp_model.generate_with_grad = MethodType(generate_with_grad, pp_model)

## Victim Model (VM)
vm_tokenizer = AutoTokenizer.from_pretrained(vm_name)
vm_model = AutoModelForSequenceClassification.from_pretrained(vm_name, local_files_only=True).to(device)
vm_idx2lbl = vm_model.config.id2label
vm_lbl2idx = vm_model.config.label2id
vm_num_labels = vm_model.num_labels

## Semantic Textual Similarity (STS) model
sts_model = SentenceTransformer(sts_name)

# Pad vocab to multiple of 8 (for better tensor core efficiency in fp16)
if pad_token_embeddings: 
    def pad_token_embeddings_to_multiple_of_n(model, n):
        def get_new_vocab_size(model): return int((np.floor(model.config.vocab_size / n) + 1) * n)
        model.resize_token_embeddings(get_new_vocab_size(model))
    pad_token_embeddings_to_multiple_of_n(pp_model, 8)
    pad_token_embeddings_to_multiple_of_n(vm_model, 8)
    # sts_model is from SentenceTransformers so needs a bit of unwrapping to access the base huggingface model 
    pad_token_embeddings_to_multiple_of_n(sts_model._first_module().auto_model, 8) 
# unlike pp_tokenizer.vocab_size this includes the padding 
vocab_size = pp_model.get_input_embeddings().num_embeddings  

### Load raw datasets and create dataloaders

#### Functions 

In [6]:
def add_idx(x, idx): x['idx'] = idx; return x   # add row numbers

def add_n_tokens(x): x['n_tokens'] = [len(o) for o in x['input_ids']]; return x  # number of tokens of "text" field 

def tokenize_fn(x):  return pp_tokenizer(x['text'], truncation=True, max_length=max_length)

def prep_small_ds(ds_dict):
    for k,v in ds_dict.items():  ds_dict[k] = v.shard(n_shards, 0, contiguous=shard_contiguous)
    return ds_dict

def get_vm_probs(text, return_predclass=False): 
    """Used also by the reward_fn to get vm_score"""
    if vm_model.training: vm_model.eval()
    with torch.no_grad():
        tkns = vm_tokenizer(text, truncation=True, padding=True, pad_to_multiple_of=padding_multiple,
                            return_tensors="pt").to(device)
        logits = vm_model(**tkns).logits
        probs = torch.softmax(logits,1)
        if return_predclass:    return probs, torch.argmax(probs,1)
        else:                   return probs

def get_vm_orig_score(batch): 
    labels = torch.tensor(batch['label'], device=device)
    orig_probs,orig_predclass = get_vm_probs(text = batch['text'], return_predclass=True)
    batch['orig_truelabel_probs'] = torch.gather(orig_probs,1, labels[:,None]).squeeze().cpu().tolist()
    batch['orig_vm_predclass'] = orig_predclass.cpu().tolist()
    return batch

def get_sampler(ds): 
    g = torch.Generator()
    g.manual_seed(seed)
    return RandomSampler(ds, generator=g)

def get_sts_orig_embeddings(batch): 
    batch['orig_sts_embeddings'] = sts_model.encode(batch['text'], batch_size=64, convert_to_tensor=False)
    return batch

def collate_fn_tkn(x): 
    d = dict()
    for k in ['idx', 'attention_mask', 'input_ids', 'label', 'orig_truelabel_probs', 'orig_sts_embeddings']: 
        d[k] = [o[k] for o in x]
    return pp_tokenizer.pad(d, pad_to_multiple_of=padding_multiple, return_tensors="pt")

def collate_fn_raw(x): 
    d = dict()
    for k in ['text', 'idx']: 
        d[k] = [o[k] for o in x]
    return d 

def get_dataloaders_dict(ds_dict, collate_fn): 
    if bucket_by_length and shuffle_train:  raise Exception("Can only do one of bucket by length or shuffle")
    d = dict()
    for split, ds in ds_dict.items(): 
        if shuffle_train:
            if split == "train": 
                sampler = get_sampler(ds)
                d[split] =  DataLoader(ds, batch_size=batch_size_train, sampler=sampler, collate_fn=collate_fn, 
                                       num_workers=n_wkrs, pin_memory=pin_memory) 
            else: 
                d[split] =  DataLoader(ds, batch_size=batch_size_eval, shuffle=False, collate_fn=collate_fn, 
                                       num_workers=n_wkrs, pin_memory=pin_memory) 
        if bucket_by_length: 
            batch_size = batch_size_train if split == "train" else batch_size_eval
            d[split] =  DataLoader(ds, batch_size=batch_size, shuffle=False, collate_fn=collate_fn, 
                                   num_workers=n_wkrs, pin_memory=pin_memory) 
    
    # Add eval dataloader for train 
    d['train_eval'] =DataLoader(ds_dict['train'], batch_size=batch_size_eval, shuffle=False,
                                collate_fn=collate_fn, num_workers=n_wkrs, pin_memory=pin_memory) 
    return d 


#### Load, tokenise, preprocess datasets 

In [7]:
if dataset_name == "rotten_tomatoes": 
    dsd_raw = load_dataset("rotten_tomatoes")
    dsd_raw['valid'] = dsd_raw.pop('validation')  # "valid" is easier than "validation"
    label_cname = 'label'
    for _,ds in dsd_raw.items():
        # make sure that all datasets have the same number of labels as what the victim model predicts
        assert ds.features[label_cname].num_classes == vm_num_labels    
elif dataset_name == "simple": 
    dsd_raw = DatasetDict()
    for s in splits:  dsd_raw[s] = load_dataset('csv', data_files=f"{path_data}simple_dataset_{s}.csv")['train']
elif dataset_name=="snli": 
    next
    ## For snli
    # remove_minus1_labels = lambda x: x[label_cname] != -1
    # ds_train = ds_train.filter(remove_minus1_labels)
    # valid = valid.filter(remove_minus1_labels)
    # test = test.filter(remove_minus1_labels)
dsd = dsd_raw.map(add_idx, batched=True, with_indices=True)
if use_small_ds: dsd = prep_small_ds(dsd)  # do after adding idx so it's consistent across runs
dsd = dsd.map(get_vm_orig_score,       batched=True)
if remove_misclassified_examples: dsd = dsd.filter(lambda x: x['orig_vm_predclass'] == x['label'])
dsd = dsd.map(get_sts_orig_embeddings, batched=True)
dsd = dsd.map(tokenize_fn, batched=True) 
dsd = dsd.map(add_n_tokens, batched=True)
if bucket_by_length: dsd = dsd.sort("n_tokens", reverse=True)
dld_raw = get_dataloaders_dict(dsd, collate_fn=collate_fn_raw) 
dld_tkn = get_dataloaders_dict(dsd, collate_fn=collate_fn_tkn)    

Using custom data configuration default-b253756c445fb811
Reusing dataset csv (/data/tproth/.cache/huggingface/datasets/csv/default-b253756c445fb811/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a)


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-c802946231f72062
Reusing dataset csv (/data/tproth/.cache/huggingface/datasets/csv/default-c802946231f72062/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a)


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-43a49c5188c42e69
Reusing dataset csv (/data/tproth/.cache/huggingface/datasets/csv/default-43a49c5188c42e69/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a)


...

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]


## Training

### Description 

Training loop pseudocode

The REINFORCE estimator is $$ \nabla_\theta J(\theta) \approx \sum_{s=1}^S  R(x,x'_s) \nabla \log p(x'_s|x,\theta)$$


$$ J(\theta) \approx \sum_{s=1}^S  R(x,x'_s) \log p(x'_s|x,\theta)$$

### Preprocessing and setup 

#### Define functions 

##### Training functions 

In [8]:
def get_entropy_metrics(scores_stacked, attention_mask): 
    ent = Categorical(logits = scores_stacked).entropy().detach()
    assert ent.shape == torch.Size([pp_batch_size, pp_length - 1])
    ent = ent * attention_mask  # stop values after eos token from contributing
    ent = (ent * attention_mask).flatten()  # remove structure (we want this across all batch values)
    ent = ent[ent != 0]   # remove the values corresponding to eos tokens from summary stats 
    assert len(ent) == len(attention_mask[attention_mask == 1].flatten())
    ent_d = dict(
        ent_min             = ent.quantile(0).item(),
        ent_lower_quartile  = ent.quantile(0.25).item(), 
        ent_median          = ent.median().item(), 
        ent_mean            = ent.mean().item(), 
        ent_upper_quartile  = ent.quantile(0.75).item(), 
        ent_max             = ent.quantile(1).item(), 
        epoch=epoch, global_step=global_step
    )
    return ent_d

def get_token_probability_metrics(scores_log_softmax, attention_mask, k=3): 
        token_prob_d = dict()
        tkn_kmaxprob, tkn_kmaxidx = torch.topk(scores_log_softmax,largest=True,  k=k, dim=2)
        tkn_kmaxprob = tkn_kmaxprob.detach()  # log these 
        assert tkn_kmaxprob.shape == torch.Size([pp_batch_size, pp_length - 1, k])

        # % of first prob over 0.9, 0.75, 0.5, 0.3, 0.1
        top_probs = tkn_kmaxprob[:,:,0].exp()
        top_probs = (top_probs * attention_mask).flatten()
        top_probs = top_probs[top_probs != 0]
        prob_threshold_l = [0.99, 0.975, 0.95, 0.90, 0.75, 0.5, 0.3, 0.1]
        for p in prob_threshold_l: 
            token_prob_d[f"top_token_prob_over_{str(p)}"] = (torch.sum(top_probs > p) / top_probs.shape[0]).item()

        # avg + median + lower + upper quartile of first, second, third choice probs
        tkn_kmaxprob_mask = tkn_kmaxprob * attention_mask[:,:,None]  # broadcasting over kth dim
        for i in range(k): 
            probs = tkn_kmaxprob_mask[:,:, i].flatten()
            probs = probs[probs != 0]
            token_prob_d[f"rank_{i+1}_token_prob_mean"] = probs.mean().item()
            token_prob_d[f"rank_{i+1}_token_prob_median"] = probs.median().item()
            token_prob_d[f"rank_{i+1}_token_prob_0.25_quantile"] = probs.quantile(0.25).item()
            token_prob_d[f"rank_{i+1}_token_prob_0.75_quantile"] = probs.quantile(0.75).item()

        # tokens over probs above 0.1, 0.01, 0.001, 0.0001, 1/vocab_size prob 
        allprobs = (scores_log_softmax.detach().exp() * attention_mask[:,:,None]).flatten()
        allprobs = allprobs[allprobs != 0]
        for p in [0.1, 0.01, 0.001, 0.0001, 0.00001]: 
            token_prob_d[f"%_of_tokens_above_prob_{p}"] =  (torch.sum(allprobs > p) / allprobs.shape[0]).item()
        token_prob_d[f"%_of_tokens_above_prob_1/vocab_size"] = \
            (torch.sum(allprobs > (1/vocab_size)) / allprobs.shape[0]).item()
            
        token_prob_d['epoch'] = epoch
        token_prob_d['global_step'] = global_step
        return token_prob_d

def get_paraphrases(input_ids,attention_mask):
    """Wrapper for generating paraphrases (pp's). Most keywords are passed on to pp_model.generate function, 
    so see docs for that function. """
    # Only greedy search supported at the moment
    pp_output = pp_model.generate_with_grad(input_ids=input_ids, 
                                            attention_mask=attention_mask, 
                                             **pp_model_params,
                                             do_sample=False, 
                                             return_dict_in_generate=True,
                                             output_scores=True,
                                                remove_invalid_values=False, 
                                             pad_token_id = pp_tokenizer.pad_token_id,
                                             eos_token_id = pp_tokenizer.eos_token_id)
    pp_l = pp_tokenizer.batch_decode(pp_output.sequences, skip_special_tokens=True)
    if track_pp_sizes:  # DEV CODE (can delete later)
        orig_max_l.append(batch['input_ids'].shape[1])
        pp_max_l.append(pp_output.sequences.shape[1])
    return pp_output, pp_l

def get_pp_logp(pp_output,log_times=True): 
    """log(p(pp|orig)) basically.
    works for greedy search, will need tweaking for other types probably"""
    ### TODO: this looks like logp to me, not plogp. Find out if this is right and if so rename, if not, fix
    ### We want to align tokens with token probabilities. The first token is given at the start 
    # and has no probability attached to it, so we remove it. 
    seq_without_first_tkn = pp_output.sequences[:, 1:]
    assert seq_without_first_tkn.shape == torch.Size([orig_batch_size, pp_length - 1])
    
    ### Convert from tuple of scores to one big tensor of scores 
    scores_stacked = torch.stack(pp_output.scores, 1)
    ### TESTS 
    # We check shape and that there is no +inf or nan in scores. 
    # Scores can have -inf in them - see explanation in `exploring_generation`.  
    assert scores_stacked.shape == torch.Size([orig_batch_size, (pp_length - 1), vocab_size])
    assert torch.all(~torch.isnan(scores_stacked))
    assert torch.all(~torch.isposinf(scores_stacked))
    # Rough check that all idx before min_length are -inf for all elements in batch
    # We do min_length - 1 because sequences are allowed to have length min_length so that idx 
    # shouldn't be set to -inf
    # Not a 100% test but very likely to identify
    idx_neginf = torch.nonzero(torch.isneginf(scores_stacked))
    assert len(idx_neginf[idx_neginf[:,2] == pp_tokenizer.eos_token_id, :]) == \
              (pp_model_params["min_length"] -1) * orig_batch_size  
    del idx_neginf
    
    ### Take log softmax of scores and then extract those that correspond 
    # to the generated sequences    
    scores_log_softmax = scores_stacked.log_softmax(2)
    seq_token_log_probs = torch.gather(scores_log_softmax,2,seq_without_first_tkn[:,:,None]).squeeze(-1)
    ### TESTS 
    # -inf is possible in scores_log_softmax and seq_token_log_probs before the attention mask is added. 
    assert torch.all(~torch.isnan(   scores_log_softmax))
    assert torch.all(~torch.isposinf(scores_log_softmax))
    check_scores_log_softmax_sums(scores_log_softmax)
    # probs should be 1-1 with the filtered tkns: check shape to confirm
    assert seq_token_log_probs.shape == seq_without_first_tkn.shape  
    # Check that the last token probability corresponds to a possible end token
    # this has to be tested before the attention mask is multiplied with it because if the 
    # padding token is 0 then this will be 0 too (and not the same as scores_log_softmax)
    output_end_ids = get_start_end_special_token_ids(pp_tokenizer)['output_end_id']
    assert all([o in scores_log_softmax[:, -1, output_end_ids] for o in seq_token_log_probs[:,-1]])
    del output_end_ids
    ## THIS ONE IS LONG - a test rather than assert 
    # check_seq_token_log_prob_values_are_correct(seq_without_first_tkn, scores_log_softmax, 
    #                                             seq_token_log_probs) 
    
    ### Generate attention mask to identify padding tokens. Then apply it to the 
    # sequence probabilities so that we don't consider probability of padding tokens 
    # when getting sequence probabilities. 
    # Also replace the -inf values in seq_token_log_probs with a large negative number because if we 
    # leave them in we end up with nan's introduced after multiplying with attention_mask, 
    # since  -inf * 0 = nan 
    attention_mask = pp_model._prepare_attention_mask_for_generation(
        seq_without_first_tkn, pp_tokenizer.pad_token_id, pp_tokenizer.eos_token_id
    )
    seq_token_log_probs = torch.nan_to_num(seq_token_log_probs, nan=None, posinf=None, neginf=-10000)
    seq_token_log_probs = seq_token_log_probs * attention_mask
    ### TESTS
    assert seq_token_log_probs.shape == attention_mask.shape == seq_token_log_probs.shape
    # check attention mask only has 0 for padding tokens and not eos tokens or anything else
    assert all(seq_without_first_tkn[attention_mask == 0] == pp_tokenizer.pad_token_id)
    check_no_nans_or_infs(seq_token_log_probs)
    # check that we aren't picking extrememly rare tokens
    assert torch.all(seq_token_log_probs  > -10)  
    
    ### Get sequence probabilities by summing up token log probabilities 
    seq_log_prob = seq_token_log_probs.sum(-1)
    ## TESTS 
    assert seq_log_prob.shape == torch.Size([pp_batch_size])
    check_no_nans_or_infs(seq_log_prob)

    
    if wandb_log_token_entropy:
        with timecode() as time_log_entropy:
            ent_d = get_entropy_metrics(scores_stacked, attention_mask)
        ent_d['time/log_entropy'] = time_log_entropy.t
        if log_times:   # need a better way to handle this. 
            wandb.log(ent_d, commit=False)
    
    
    if wandb_log_token_probabilities: 
        with timecode() as time_log_token_probabilities:
            token_prob_d = get_token_probability_metrics(scores_log_softmax, attention_mask, k=3)
        token_prob_d['time/log_token_probabilities'] = time_log_token_probabilities.t
        if log_times: 
            wandb.log(token_prob_d, commit=False)
        
    return seq_log_prob

def reward_fn(data, raw, pp_l, return_components=False, log_times=True): 
    """orig_l, pp_l are lists of original and paraphrase respectively"""
    # Victim model probability differences between orig and pp
    with timecode() as time_vm_scores:
        pp_probs = get_vm_probs(pp_l) 
        pp_predclass = torch.argmax(pp_probs, axis=1)
        pp_truelabel_probs   = torch.gather(pp_probs, 1, data['label'][:,None]).squeeze()
        pp_predclass_probs   = torch.gather(pp_probs, 1, pp_predclass[ :,None]).squeeze()
        label_flip = (pp_predclass != data['label']) * 1
        vm_scores = (data['orig_truelabel_probs'] - pp_truelabel_probs)
    
    
    # STS scores
    with timecode() as time_sts_scores:
        pp_embeddings   = sts_model.encode(pp_l,        batch_size=len(raw), convert_to_tensor=True, device=device)
        # This returns a cosine similarity matrix, of which we just want the diagonal
        sts_scores = pytorch_cos_sim(data['orig_sts_embeddings'], pp_embeddings).diagonal()  
    
    # Reward calculation 
    rewards = torch.tensor([-0.5 if sts < 0.5 else 0.5+v*sts for v,sts in zip(vm_scores, sts_scores)],device=device)
    
    if log_times:
        wandb.log({'epoch': epoch, 'global_step': global_step, 
                   'time/vm_scores': time_vm_scores.t, 'time/sts_scores': time_sts_scores.t }, 
                   commit=False)
    
    if return_components: 
        return {
            "orig_l": raw['text'],
            "pp_l": pp_l,  
            "truelabel": data['label'],
            "orig_truelabel_probs": data['orig_truelabel_probs'],
            "pp_truelabel_probs":  pp_truelabel_probs,
            "pp_predclass": pp_predclass,
            "pp_predclass_probs": pp_predclass_probs,
            "vm_score": vm_scores, 
            "sts_score": sts_scores,
            "reward": rewards,
            "label_flip": label_flip
        }
    else:  return {"reward": rewards}

def pp_model_forward(data): 
    global orig_batch_size,orig_length,pp_batch_size,pp_length
    orig_batch_size     = data['input_ids'].shape[0]
    orig_length         = data['input_ids'].shape[1]
    pp_output, pp_l = get_paraphrases(data['input_ids'], data['attention_mask']) 
    pp_batch_size = pp_output.sequences.shape[0]
    # for greedy search pp_length is equal to orig_batch_size but this won't be for beam search
    pp_length     = pp_output.sequences.shape[1]  
    return pp_output, pp_l

def loss_fn(data, raw, pp_output, pp_l, return_components=False, log_times=True): 
    with timecode() as time_reward_fn:
        d = reward_fn(data, raw, pp_l, return_components=return_components, log_times=log_times)
        
    if normalise_rewards: 
        d['orig_reward'] = copy.deepcopy(d['reward'])
        d['reward'] = (d['reward']-torch.mean(d['reward']))/torch.std(d['reward'])
        
    with timecode() as time_pp_logp:
        d['pp_logp'] = get_pp_logp(pp_output,log_times=log_times)
    
    with timecode() as time_loss_fn_loss_calc:
        d['loss'] = -d['reward'] * d['pp_logp']
        d['loss_batch'] = torch.mean(d['loss'])
        if return_components ==  False: return d['loss_batch'] 
        
    # remove some items from compgraph
    with timecode() as time_loss_fn_detach:
        d['pp_logp'] = d['pp_logp'].detach()  
        d['loss']    = d['loss'].detach()
        
#     # This was taking a lot of time so removed it. Add it in if needed. 
#     #gc.collect() 
#     print("\t### INSIDE loss_fn### ")
#     print("\tglobal_step", global_step)
#     print("\treward_fn_time", time_reward_fn.t)
#     print("\t######### ")

    if log_times:   # true for training, not eval. 
        wandb.log({'epoch': epoch, 'global_step': global_step, 
                   'time/reward_fn': time_reward_fn.t, 'time/pp_logp': time_pp_logp.t, 
                  'time/loss_fn_loss_calc': time_loss_fn_loss_calc.t, 
                   'time/pp_logp_detach': time_loss_fn_detach.t, 
                  }, 
                 commit=False)
    return d

def training_step(data, raw, accelerator): 
    """With gradient accumulation"""
    with timecode() as time_generate_pp:
        pp_output, pp_l = pp_model_forward(data)
    
    #logger.info(show_gpu(f'Batch {batch_num}, GPU memory usage after forward pass: '))
    
    with accelerator.autocast():
        with timecode() as time_loss_fn:
            if wandb_log_training_step_table: 
                results_d = loss_fn(data, raw, pp_output, pp_l, return_components=True)
                loss_batch = results_d['loss_batch']
            else: 
                loss_batch = loss_fn(data, raw, pp_output, pp_l, return_components=False)
        
        loss_batch = loss_batch / accumulation_steps  # Normalize our loss for gradient accumulation
        
    with timecode() as time_backwards:
        accelerator.backward(loss_batch) 
    
    #logger.info(show_gpu(f'Batch {batch_num}, GPU memory usage after backwards pass: '))
    if (accumulation_num + 1) % accumulation_steps == 0: 
        with timecode() as time_opt_step:
            optimizer.step()
        
        pp_model.zero_grad(set_to_none=zero_grad_with_none)
    if wandb_log_training_step_table: 
        with timecode() as time_add_to_training_step_table:
            results_d = process_results_d1(results_d, raw)
            add_preds_to_data_d(results_d, split='training_step') 
        
#     print("### INSIDE training_step ####")    
#     print("epoch", epoch)
#     print("batch_num", batch_num)
#     print("global_step", global_step)
#     print("model in training mode", pp_model.training)
#     print("orig", raw['text'])
#     print("pp_l", pp_l)
#     print("pp_seq", pp_output.sequences)
#     print("pp_length", pp_output.sequences.shape, pp_length)
#     print("loss_fn_time", time_loss_fn.t)
#     print("### INSIDE training_step ####")
    
    wandb.log({'time/generate_pp': time_generate_pp.t, 'time/loss_fn': time_loss_fn.t, 
               'time/backwards_pass': time_backwards.t, 'time/optimizer_step': time_opt_step.t, 
               'time/add_to_training_step_table': time_add_to_training_step_table.t, 
               'epoch': epoch, 'global_step': global_step,'batch_num': batch_num,
               'orig_length': orig_length,'orig_batch_size': orig_batch_size,
              'pp_length': pp_length, 'pp_batch_size': pp_batch_size}
              ,commit=False)

def process_results_d1(results_d, raw): 
    """REFACTOR THIS LATER"""
    # wandb logging 
    results_d['epoch'] = epoch
    results_d['idx'] = raw['idx']
    for k,v in results_d.items(): 
        if torch.is_tensor(v): 
            results_d[k] = v.detach().cpu().tolist()
        elif type(v) == int or type(v) == float: 
            # make into list repeated n times
            results_d[k] = [v for o in range(batch_size_train)]
    return results_d

def save_model(): 
    path = f"{path_run}model_{epoch}"
    torch.save({
                'epoch': epoch,
                'pp_model_state_dict': pp_model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                }, path)
    
def resume_model(path): 
    state = torch.load(path)
    pp_model.load_state_dict(state['pp_model_state_dict'])
    optimizer.load_state_dict(state['optimizer_state_dict'])
    

##### Eval and wandb functions 

In [9]:
def table2df(table):  return pd.DataFrame(data=table.data, columns=table.columns)  # wandb table to dataframe

def process_results_d_for_wandb(results_d): 
    # Flatten batches for each key, depending on datatype (e.g. lists of lists )
    for k,v in results_d.items(): 
        # v[0] is arbitrary - we are just checking the first item in the list to see the type
        if type(v) == float or type(v) == int: 
            next
        elif  torch.is_tensor(v[0]): 
            # case where we have a list of scalars - the cat function doesn't work here 
            if  v[0].size() == torch.Size([]): x = torch.stack(v)
            else:                              x = torch.cat(v)
            results_d[k] = x.detach().cpu().squeeze().tolist()  # convert to list (squeeze is for single scalar list)
        elif type(v[0]) == list:  # this is True for tensors also, so it has to go after the is_tensor check
            results_d[k] = list(itertools.chain(*v)) 
        elif type(v) == list: 
            next
        else: 
            raise Exception("shouldn't get here")
    return results_d

def eval_dl(dl_tkn, dl_raw, device): 
    """Get evaluation metrics for a dataloader"""
    # Put models in eval mode and do the forward pass 
    # Current logic: push all batches together into one big list.     
    if pp_model.training: pp_model.eval()
    if vm_model.training: vm_model.eval()
    results_d = defaultdict(list)
    with torch.no_grad(): 
        for eval_batch_num, (data, raw) in enumerate(zip(dl_tkn, dl_raw)):
          #  logger.info(show_gpu(f'EVAL, batch {i}, GPU memory usage after loading data: '))
            for k, v in data.items(): 
                if data[k].device != device: data[k] = data[k].to(device)
           # if data['input_ids'].device != device: data['input_ids'].to(device)
                
            pp_output, pp_l = pp_model_forward(data)
            d = loss_fn(data, raw, pp_output, pp_l, return_components=True, log_times=False)
            #logger.info(show_gpu(f'EVAL, batch {eval_batch_num}, GPU memory usage after loss_fn pass: '))
            d['idx'] = raw['idx']
                      
            for k,v in d.items(): 
                results_d[k].append(v) 
    del eval_batch_num, data, raw, pp_output, pp_l, d
    results_d = process_results_d_for_wandb(results_d)
            
    # Calculate additional metrics 
    results_d['epoch'] = epoch
    return results_d

def add_preds_to_data_d(results_d, split):
    if split not in data_d.keys() or split == "training_summary": # training summary table logic is elsewhere
        raise Exception("split not in table keys or split == training_summary ") 
    #table = table_d[split]
        
    # Need epoch to be repeated to the same length as the rest of the fields 
    # (this isn't the batch size because we concat a bunch of stuff)
    # we don't want to change the `epoch` key because it screws up logging of the other metrics. 
    # So we make a new dict.
    # d1 = copy.deepcopy(results_d)
    d1 = results_d
    d1['epoch'] = [epoch for o in range(len(d1['pp_l']))]
    dcols = [d1[c] for c in table_columns]  # filter out loss_batch
    assert len(set([len(o) for o in dcols])) == 1  # all lists should be of the same length 
    
    for row in zip(*dcols):
        d2 = {k:v for k,v in zip(table_columns,row)}
        data_d[split].append(d2)

def update_training_summary_table(results_d, split):
    d = dict()
    # key names here have to match those in summary_table_columns
    d['epoch'] = epoch
    d['split'] = split
    for metric in metrics:
        d[f'{metric}_avg'] = np.mean(results_d[metric])
    #data_d['training_summary'].append(*[d[c] for c in summary_table_columns])
    data_d['training_summary'].append(d)

def log_wandb_tables(run): 
    """Log wandb tables to the UI"""
    d = dict()
    d["eval/training_summary_table"] = table_d['training_summary']
  #  print(len(d["eval/training_summary_table"].data))
    run.log(d)

def plot_examples_chart(split, table, metric, commit):
    spec = "uts_nlp/line_chart_v2"
    fields = {"x": "epoch",'groupKeys': 'idx'}
    fields['y'] = f"{metric}"
    string_fields = dict()
    string_fields['title'] = f"{split}_{metric} vs epoch (examples)"
    chart = wandb.plot_table(vega_spec_name=spec, data_table=table, 
                            fields=fields, string_fields=string_fields)
    wandb.log({f"individual_examples/{split}_{metric}_vs_epoch_examples": chart}, commit=commit)

def plot_summary_charts(metric, table, commit):
    spec = "uts_nlp/line_chart_v2"
    fields = {"x": "epoch",'groupKeys': 'split'}
    fields['y']  = f"{metric}_avg"
    chart = wandb.plot_table(vega_spec_name=spec, data_table=table, 
                                 fields=fields, string_fields={'title': f"{metric} vs epoch"})
    
    wandb.log({f"summary_charts/avg_{metric}_vs_epoch": chart}, commit=commit)

def plot_wandb_charts(): 
    if wandb_plot_examples: 
        # Examples charts 
        for split in ['train', 'valid']:
            df = pd.DataFrame(data_d[split]) if type(data_d[split]) is list else data_d[split]
            df = df.query("idx in @plt_idx_d[@split]").sort_values(['idx', 'epoch'])
            for metric in metrics: 
                plot_examples_chart(split, table=wandb.Table(dataframe=df), metric=metric, commit=False)
        
    ## Summary charts 
    for metric in metrics: 
        commit = True if metric == metrics[len(metrics)-1] else False
        df = pd.DataFrame(data_d['training_summary'])
        plot_summary_charts(metric, table=wandb.Table(dataframe=df),
                            commit=commit)

def add_wandb_run_summary_statistics(run):
    ## Training summary statistics 
    df_summary = pd.DataFrame(data_d['training_summary']) 
    # We calculate the best epoch according to the validation set
    best_epoch_idx = df_summary.query("split=='valid'")['loss_avg'].idxmin() 
    valid_row = df_summary.iloc[best_epoch_idx]
    best_epoch = valid_row['epoch'].item()
    run.summary['best_epoch'] = best_epoch
    # iloc transforms 1row df to series (so it is same as  valid_row)
    train_row = df_summary.query("split=='train' & epoch==@best_epoch").iloc[0]  
    for metric in metrics: 
        run.summary[f"{metric}_avg_train"] = train_row[f"{metric}_avg"].item()
        run.summary[f"{metric}_avg_valid"] = valid_row[f"{metric}_avg"].item()
                                 
    ## Summary statistics of the test set 
    # From the last epoch atm because we don't have early stopping 
    test_metrics = data_d['test'].filter(metrics, axis=1).mean()
    for metric, val in zip(test_metrics.index, test_metrics): 
        run.summary[f"{metric}_avg_test"] = val

##### Tests and asserts for `get_pp_logp`

In [10]:
def get_start_end_special_token_ids(tokenizer): 
    """The token id's that input/output sequences should start and end with"""
    d = {}
    if tokenizer.name_or_path in ['eugenesiow/bart-paraphrase', 'tdopierre/ProtAugment-ParaphraseGenerator']: 
        d["input_start_id"] =  tokenizer.bos_token_id
        d["input_end_id"] =  [tokenizer.pad_token_id, tokenizer.eos_token_id]
        d["output_start_id"] =  tokenizer.eos_token_id 
        d["output_end_id"] =  [tokenizer.pad_token_id, tokenizer.eos_token_id]
    elif tokenizer.name_or_path == "tuner007/pegasus_paraphrase":
        d["input_start_id"] =  None
        d["input_end_id"] =  [tokenizer.pad_token_id, tokenizer.eos_token_id] 
        d["output_start_id"] =  tokenizer.pad_token_id
        d["output_end_id"] =  [tokenizer.pad_token_id, tokenizer.eos_token_id]
    else: 
        raise Exception("unrecognised tokenizer")
    return d

def check_no_nans_or_infs(x):
    assert torch.all(~torch.isnan(x))
    assert torch.all(~torch.isneginf(x))
    assert torch.all(~torch.isposinf(x))
    
def assert_start_and_end_tokens_are_correct(tokenizer, orig_token_ids, pp_token_ids):
    """Make sure input sequences (orig) and output sequences (pp) start and end with the 
    right special tokens (depends on tokenizer)"""
    start_end_token_d = get_start_end_special_token_ids(pp_tokenizer)
    
    # Input
    if start_end_token_d['input_start_id'] is not None: 
        assert torch.all(orig_token_ids[:,0] == start_end_token_d['input_start_id'])
    # can probs rewrite this to make it nicer but it's fine for now
    assert torch.all(torch.logical_or(orig_token_ids[:,-1] == start_end_token_d['input_end_id'][0], 
                                      orig_token_ids[:,-1] == start_end_token_d['input_end_id'][1]))
    
    # Output
    assert torch.all(pp_token_ids[:,0] == start_end_token_d['output_start_id'])
    assert torch.all(torch.logical_or(pp_token_ids[:,-1] == start_end_token_d['output_end_id'][0], 
                                      pp_token_ids[:,-1] == start_end_token_d['output_end_id'][1]))
    
def check_scores_log_softmax_sums(scores_log_softmax):
    sums = scores_log_softmax.exp().sum(2)
    # check that the axes is right
    # we want to sum over token probabilities at each generation step, so we 
    # should end up with a shape [orig_batch_size, pp_length]
    assert sums.shape[0] == orig_batch_size  
    assert sums.shape[1] == pp_length - 1
    # check that they sum to 1 along the pp_length axis
    assert torch.allclose(sums, torch.ones(sums.size(), device=device), atol = 1e-4)
    
def check_seq_token_log_prob_values_are_correct(seq_without_first_tkn, scores_log_softmax, seq_token_log_probs): 
    """Just enumerates and checks values
    Quite slow for large batches so run as a test rather than an assert in every batch. 
    """
    l = []
    for i_ex in range(orig_batch_size):
        for i_step in range(pp_length - 1):
            i_tkn = seq_without_first_tkn[i_ex][i_step].item()
            l.append(scores_log_softmax[i_ex,i_step, i_tkn] == seq_token_log_probs[i_ex,i_step])
    assert all(l)    

In [11]:
#### Set up models and do layer freezing
vm_model.eval()
pp_model.train()
## Layer freezing 
# Unfreeze last 2 layers of the base model decoder
# Not sure if decoder layer norm should be unfrozen or not, but it appears after the
#   other parameters in the module ordering, so let's include it for now
# Also unfreeze the linear head.  This isn't stored in the base model but rather tacked on top
#   and will be fine-tuned for summarisation. 
if pp_name == "tuner007/pegasus_paraphrase":
    layer_list = ['decoder.layers.14', 'decoder.layers.15', 'decoder.layer_norm'] 
elif pp_name == "tdopierre/ProtAugment-ParaphraseGenerator":
    layer_list = ['decoder.layers.4','decoder.layers.5', 'decoder.layernorm_embedding']
elif pp_name == "eugenesiow/bart-paraphrase":
    layer_list = ['decoder.layers.10','decoder.layers.11', 'decoder.layernorm_embedding']
for i, (name,param) in enumerate(pp_model.base_model.named_parameters()): 
    if np.any([o in name for o in layer_list]):   param.requires_grad = True
    else:                                         param.requires_grad = False
for param in pp_model.lm_head.parameters():       param.requires_grad = True
# Not sure if to include this or not. this seems to affect lm_head. i might just leave it as it was for now.
# this will freeze the embeddings/lm head. 
# From here: https://github.com/huggingface/transformers/issues/10479#issuecomment-788964822
# self.lm_head is tied (the same parameter as) to self.encoder.embed_tokens and self.decoder.embed_tokens.
for param in pp_model.base_model.shared.parameters(): param.requires_grad = False 
### For checking the grad status of the layers
# for i, (name, param) in enumerate(pp_model.base_model.named_parameters()): print(i, name, param.requires_grad)
# for i, (name, param) in enumerate(pp_model.lm_head.named_parameters()):    print(i, name, param.requires_grad)

#### Optimizer
# For now we just keep this simple
optimizer = AdamW(pp_model.parameters(), lr=lr)

#### Set up other miscellaneous things
#rouge_metric = load_metric("rouge", keep_in_memory=True)
n_train_steps = n_train_epochs * len(dld_tkn['train'])

/home/tproth/Programs/miniconda/envs/nlp_env/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [12]:
def training_function(pp_model, vm_model, dld_tkn, dld_raw, optimizer): 
    accelerator = Accelerator()
    device = accelerator.device
    
    pp_model,vm_model,optimizer,dld_tkn['train'] = accelerator.prepare(
        pp_model,vm_model,optimizer,dld_tkn['train']
    )
    
    #logger.info(show_gpu(f' GPU memory usage after loading models:'))
    progress_bar = tqdm(range(n_train_steps))

    pp_model.zero_grad(set_to_none=zero_grad_with_none) 
    global accumulation_num, global_step, epoch, batch_num
    accumulation_num = 0 
    global_step = 0
    for epoch in range(n_train_epochs): 
        logger.info(f"Now on epoch {epoch} of {n_train_epochs}")
        if not pp_model.training: pp_model.train()
        with timecode() as time_train_one_epoch:
            for batch_num, (data, raw) in enumerate(zip(dld_tkn['train'], dld_raw['train'])): 
                if batch_num % 10 == 0 :   logging.info(f"Now processing batch {batch_num} out of {len(dld_tkn['train'])}")
                training_step(data, raw, accelerator) 
                accumulation_num += 1
                progress_bar.update(1) 
                global_step += 1 
        
        wandb.log({'time/train_one_epoch_time': time_train_one_epoch.t,
                   'time/train_one_epoch_thoroughput': len(dsd['train']) / time_train_one_epoch.t,
                   'epoch': epoch}, commit=True)
        
        if wandb_log_grads and epoch % wandb_log_grads_freq == 0: 
            plt = plot_grad_flow(pp_model.named_parameters())
            wandb.log({"gradient flow": wandb.Image(plt)})  # doesn't work as a non-image (i.e. plotly)
            del plt 
        gc.collect() 
        torch.cuda.empty_cache()
        
        if save_model_while_training and (epoch + 1) % save_model_freq == 0:  save_model(epoch)
        
        # Evaluation loop
        if epoch % eval_freq == 0: 
            logging.info(f"Now doing train eval")
            with timecode() as time_eval_train:
                train_set_preds = eval_dl(dl_tkn = dld_tkn['train_eval'], dl_raw=dld_raw['train_eval'], 
                                          device=device)

            logging.info(f"Now doing valid eval")
            with timecode() as time_eval_valid:
                valid_set_preds = eval_dl(dl_tkn = dld_tkn['valid'],      dl_raw=dld_raw['valid'], 
                                        device=device)
            
            # update the tables every epoch and log them
            with timecode() as time_update_training_summary_table:
                update_training_summary_table(train_set_preds, split='train')
                update_training_summary_table(valid_set_preds, split='valid')
            with timecode() as time_add_eval_preds_to_data_d:    
                add_preds_to_data_d(train_set_preds, split='train')
                add_preds_to_data_d(valid_set_preds, split='valid')
            #log_wandb_tables(run)
            plot_wandb_charts()
            del train_set_preds
            del valid_set_preds
            with timecode() as time_eval_gc_collect:
                gc.collect() 
            with timecode() as time_eval_empty_cache:
                torch.cuda.empty_cache()
            wandb.log({'time/eval_train_time': time_eval_train.t, 'time/eval_valid_time': time_eval_valid.t,
                       'time/eval_train_thoroughput': len(dsd['train']) / time_eval_train.t,
                       'time/eval_valid_thoroughput': len(dsd['valid']) / time_eval_valid.t,
                       'time/eval_update_training_summary_table': time_update_training_summary_table.t, 
                       'time/eval_add_preds_to_data_d': time_add_eval_preds_to_data_d.t,
                       'time/eval_gc_collect': time_eval_gc_collect.t, 
                       'time/eval_empty_cache': time_eval_empty_cache.t,
               'epoch': epoch},
                      commit=True)
    
    logging.info(f"Now doing test eval")        
    # Eval on test set 
    test_set_preds = eval_dl(dl_tkn = dld_tkn['test'], dl_raw=dld_raw['test'],  device=device)
    add_preds_to_data_d(test_set_preds, split='test')
    # Log, plot, and finish up
    #log_wandb_tables(run)
    
    # Data -> df and save dfs to file 
    for key in data_d.keys():  # splits and sometimes 'training_step' too 
        data_d[key] = pd.DataFrame(data_d[key]) # dict of list of dict -> dict of dataframe
        data_d[key].to_csv(f"{path_run}{key}.csv", index=False)
    # Save training_summary table to csv too 
    pd.DataFrame(data_d['training_summary']).to_csv(f"{path_run}training_summary.csv", index=False)
        
    plot_wandb_charts()
    add_wandb_run_summary_statistics(run)

#### Run

In [13]:
#resume_model(f"{path_checkpoints}devout-durian-172_39")
## Launch run and configure what it is tracking
#wandb_mode='disabled'
#wandb_mode='online'
run = wandb.init(project="travis_attack", entity="uts_nlp", config=config_d,
                 mode=wandb_mode, notes=run_notes)
if wandb_log_grads: wandb.watch(pp_model, log='gradients', log_freq=wandb_log_grads_freq)

path_run = f"{path_checkpoints}{run.name}/"
if not os.path.exists(path_run): os.makedirs(path_run, exist_ok=True)
### Set up tables 


##### TO REFACTOR ######
# These have to be in the keys of the output from eval_dl
# table_columns = ['idx', 'orig_l',  'truelabel', 'orig_truelabel_probs', 'epoch', 'pp_l',
#              'pp_truelabel_probs', "pp_predclass", "pp_predclass_probs"] + metrics
# def make_table(cols): return wandb.Table(columns=cols)
# for key in splits:                table_d[key]             = make_table(table_columns) 
# if wandb_log_training_step_table: table_d['training_step'] = make_table(table_columns) 
# summary_table_columns = ['epoch','split'] + [f'{m}_avg' for m in metrics]
# table_d['training_summary'] = make_table(summary_table_columns)
########################


#### NEW ####
## Raw observation data (lists of dicts, later becomes pandas df)
# These have to be in the keys of the output from eval_dl
table_columns = ['idx', 'orig_l',  'truelabel', 'orig_truelabel_probs', 'epoch', 'pp_l',
             'pp_truelabel_probs', "pp_predclass", "pp_predclass_probs"] + metrics
summary_table_columns = ['epoch','split'] + [f'{m}_avg' for m in metrics]

data_d = dict()
for key in splits + ['training_summary']:  data_d[key]             = [] 
if wandb_log_training_step_table:          data_d['training_step'] = []

## Training summary table (W&B table)
# Future W&B tables here
#table_d = dict()
#def make_table(cols): return wandb.Table(columns=cols)
#table_d['training_summary'] = make_table(summary_table_columns)
#############

## Get indices for the examples plots
if wandb_plot_examples:
    def get_examples_plot_idxs(ds): 
        return np.random.choice(ds['idx'], size=wandb_n_examples_plot, replace=False).tolist()
    plt_idx_d = dict()
    for split in splits: plt_idx_d[split] = get_examples_plot_idxs(dsd[split])

#%lprun -f training_function -f  get_pp_logp -f training_step -f  reward_fn -f  loss_fn -f eval_dl  notebook_launcher(training_function, args=(pp_model, vm_model, dld_tkn, dld_raw, optimizer), num_processes=1, use_fp16=use_fp16)
notebook_launcher(training_function, args=(pp_model, vm_model, dld_tkn, dld_raw, optimizer), num_processes=1, use_fp16=use_fp16)


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


wandb: Currently logged in as: uts_nlp (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.10 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Launching training on one GPU.


  0%|          | 0/120 [00:00<?, ?it/s]

Now on epoch 0 of 60
	### INSIDE Reward_fn### 
	global_step 0
	reward_fn_time 0.01882677525281906
	######### 
### INSIDE training_step ####
epoch 0
batch_num 0
global_step 0
model in training mode True
orig ['I do not like this movie', 'I hate this apple']
pp_l ['What do you think about this movie?', 'I hate this apple.']
pp_seq tensor([[    2,  2264,   109,    47,   206,    59,    42,  1569,   116,     2],
        [    2,   100,  4157,    42, 15162,     4,     2,     1,     1,     1]],
       device='cuda:0')
pp_length torch.Size([2, 10]) 10
loss_fn_time 0.027931813150644302
### INSIDE training_step ####
	### INSIDE Reward_fn### 
	global_step 1
	reward_fn_time 0.017211955040693283
	######### 
### INSIDE training_step ####
epoch 0
batch_num 1
global_step 1
model in training mode True
orig ['I love this apple', 'I like this movie']
pp_l ['I love this apple.', 'What do you think about this movie?']
pp_seq tensor([[    2,   100,   657,    42, 15162,     4,     2,     1,     1,     1],
   

...

Now on epoch 27 of 60
	### INSIDE Reward_fn### 
	global_step 54
	reward_fn_time 0.01896532252430916
	######### 
### INSIDE training_step ####
epoch 27
batch_num 0
global_step 54
model in training mode True
orig ['I do not like this movie', 'I hate this apple']
pp_l ['What do you think about this movie?', 'What do you hate about this apple?']
pp_seq tensor([[    2,  2264,   109,    47,   206,    59,    42,  1569,   116,     2],
        [    2,  2264,   109,    47,  4157,    59,    42, 15162,   116,     2]],
       device='cuda:0')
pp_length torch.Size([2, 10]) 10
loss_fn_time 0.026142582297325134
### INSIDE training_step ####
	### INSIDE Reward_fn### 
	global_step 55
	reward_fn_time 0.01678711175918579
	######### 
### INSIDE training_step ####
epoch 27
batch_num 1
global_step 55
model in training mode True
orig ['I love this apple', 'I like this movie']
pp_l ['What do you love about this apple?', 'What do you think about this movie?']
pp_seq tensor([[    2,  2264,   109,    47,   657,  

	### INSIDE Reward_fn### 
	global_step 67
	reward_fn_time 0.016845203936100006
	######### 
### INSIDE training_step ####
epoch 33
batch_num 1
global_step 67
model in training mode True
orig ['I love this apple', 'I like this movie']
pp_l ['What do you love about this apple?', 'What do you think about this movie?']
pp_seq tensor([[    2,  2264,   109,    47,   657,    59,    42, 15162,   116,     2],
        [    2,  2264,   109,    47,   206,    59,    42,  1569,   116,     2]],
       device='cuda:0')
pp_length torch.Size([2, 10]) 10
loss_fn_time 0.023494407534599304
### INSIDE training_step ####
Now on epoch 34 of 60
	### INSIDE Reward_fn### 
	global_step 68
	reward_fn_time 0.018516726791858673
	######### 
### INSIDE training_step ####
epoch 34
batch_num 0
global_step 68
model in training mode True
orig ['I do not like this movie', 'I hate this apple']
pp_l ['What do you think about this movie?', 'What do you hate about this apple?']
pp_seq tensor([[    2,  2264,   109,    47,   206,

	### INSIDE Reward_fn### 
	global_step 81
	reward_fn_time 0.018142245709896088
	######### 
### INSIDE training_step ####
epoch 40
batch_num 1
global_step 81
model in training mode True
orig ['I love this apple', 'I like this movie']
pp_l ['What do you love about this apple?', 'What do you think about this movie?']
pp_seq tensor([[    2,  2264,   109,    47,   657,    59,    42, 15162,   116,     2],
        [    2,  2264,   109,    47,   206,    59,    42,  1569,   116,     2]],
       device='cuda:0')
pp_length torch.Size([2, 10]) 10
loss_fn_time 0.025161463767290115
### INSIDE training_step ####
	### INSIDE Reward_fn### 
	global_step 82
	reward_fn_time 0.027051199227571487
	######### 
	### INSIDE Reward_fn### 
	global_step 82
	reward_fn_time 0.026211507618427277
	######### 
Now on epoch 41 of 60
	### INSIDE Reward_fn### 
	global_step 82
	reward_fn_time 0.017322465777397156
	######### 
### INSIDE training_step ####
epoch 41
batch_num 0
global_step 82
model in training mode True
orig [

Now on epoch 47 of 60
	### INSIDE Reward_fn### 
	global_step 94
	reward_fn_time 0.018363341689109802
	######### 
### INSIDE training_step ####
epoch 47
batch_num 0
global_step 94
model in training mode True
orig ['I do not like this movie', 'I hate this apple']
pp_l ['What do you think about this movie?', 'What do you hate about this apple?']
pp_seq tensor([[    2,  2264,   109,    47,   206,    59,    42,  1569,   116,     2],
        [    2,  2264,   109,    47,  4157,    59,    42, 15162,   116,     2]],
       device='cuda:0')
pp_length torch.Size([2, 10]) 10
loss_fn_time 0.02584323287010193
### INSIDE training_step ####
	### INSIDE Reward_fn### 
	global_step 95
	reward_fn_time 0.01680738478899002
	######### 
### INSIDE training_step ####
epoch 47
batch_num 1
global_step 95
model in training mode True
orig ['I love this apple', 'I like this movie']
pp_l ['What do you love about this apple?', 'What do you think about this movie?']
pp_seq tensor([[    2,  2264,   109,    47,   657,  

	### INSIDE Reward_fn### 
	global_step 107
	reward_fn_time 0.016950063407421112
	######### 
### INSIDE training_step ####
epoch 53
batch_num 1
global_step 107
model in training mode True
orig ['I love this apple', 'I like this movie']
pp_l ['What do you love about this apple?', 'What do you think about this movie?']
pp_seq tensor([[    2,  2264,   109,    47,   657,    59,    42, 15162,   116,     2],
        [    2,  2264,   109,    47,   206,    59,    42,  1569,   116,     2]],
       device='cuda:0')
pp_length torch.Size([2, 10]) 10
loss_fn_time 0.023600667715072632
### INSIDE training_step ####
Now on epoch 54 of 60
	### INSIDE Reward_fn### 
	global_step 108
	reward_fn_time 0.01966610550880432
	######### 
### INSIDE training_step ####
epoch 54
batch_num 0
global_step 108
model in training mode True
orig ['I do not like this movie', 'I hate this apple']
pp_l ['What do you think about this movie?', 'What do you hate about this apple?']
pp_seq tensor([[    2,  2264,   109,    47,   2

In [14]:
%debug

No traceback has been produced, nothing to debug.


In [15]:
run.finish()

%_of_tokens_above_prob_0.0001,█▇▄▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
%_of_tokens_above_prob_0.001,█▇▄▃▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
%_of_tokens_above_prob_0.01,█▇▄▄▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
%_of_tokens_above_prob_0.1,█▅▂▃▃▃▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
%_of_tokens_above_prob_1/vocab_size,█▇▄▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
%_of_tokens_above_prob_1e-05,█▇▄▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
batch_num,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
ent_lower_quartile,█▇▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
ent_max,▇█▆▆▃▄▂▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
ent_mean,█▆▄▃▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
ent_median,█▆▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁


In [16]:
break

SyntaxError: 'break' outside loop (<ipython-input-16-6aaf1f276005>, line 1)

In [ ]:
import pandas as pd, numpy as np, seaborn as sns
import matplotlib.pyplot as plt
import wandb
from datasets import load_metric, Dataset
from IPython.display import Markdown, display
from IPython.core.debugger import set_trace
import plotly.express as px

from datasets import load_dataset, Dataset

from lexicalrichness import LexicalRichness
import functools
import string
import psutil
from collections import defaultdict
from spacy_wordnet.wordnet_annotator import WordnetAnnotator 
import operator
import spacy
import textstat
import difflib as dl
from itertools import groupby
from operator import itemgetter
from analysis_functions import *

In [ ]:
pd.options.mode.chained_assignment = None  # default='warn'
run_name = run.name
run_id = run.id
# TODO: merge this with the other path 
path_run_results = f"../model_checkpoints/travis_attack/{run_name}/"
train            = pd.read_csv(path_run_results + "train.csv")
valid            = pd.read_csv(path_run_results + "valid.csv")
#test             = pd.read_csv(path_run_results + "test.csv")
training_step    = pd.read_csv(path_run_results + "training_step.csv")
#training_summary = pd.read_csv(path_run_results + "training_summary.csv")

In [ ]:
num_proc = 2
df_training_step = postprocess_df(training_step, filter_idx=None, num_proc=num_proc)
df_train = postprocess_df(train, filter_idx=None, num_proc=num_proc)
df_valid = postprocess_df(valid, filter_idx=None,  num_proc=num_proc)

In [ ]:
df_training_step.to_pickle(path_run_results + "df_temp_training_step.pkl")
df_train.to_pickle(path_run_results + "df_temp_train.pkl")
df_valid.to_pickle(path_run_results + "df_temp_valid.pkl")
# df_training_step = pd.read_pickle(path_run_results + "df_temp_training_step.pkl")
# df_train = pd.read_pickle(path_run_results + "df_temp_train.pkl")
# df_valid = pd.read_pickle(path_run_results + "df_temp_valid.pkl")

In [ ]:
df_training_step['data_split'] = 'training_step'
df_train['data_split'] = 'eval_train'
df_valid['data_split'] = 'eval_valid'
df_concat = pd.concat([df_training_step,df_train,df_valid])
df_concat = df_concat.reset_index(drop=True)
# Otherwise we get a big spike at 0
df_concat.loc[df_concat.epoch_of_first_label_flip == 0, 'epoch_of_first_label_flip'] = None
fig_l = []

hist_config_dicts = [
    {
        'colname': 'epoch_of_first_label_flip', 
        'xlabel': "Epoch of first label flip", 
        'desc': "Cumulative prob Epoch of first label flip for each original example",
        'cumulative': True,
    },
    {
        'colname': 'idx_n_unique_pp', 
        'xlabel': "Unique paraphrases per original example", 
        "desc": "Number of generated unique paraphrases per original example during training", 
        'cumulative': False,
    },
    {
        'colname': 'idx_n_pp_changes', 
        'xlabel': "Paraphrase changes per original example", 
        "desc": "Number of paraphrase changes per original example during training", 
        'cumulative': False,
    },
]


fig_l = []
line_colnames = [o for o in df_concat.columns if "_diff" in o] + [
    'rouge_score', "is_truncation", 'any_phrase_capitalised', 'any_phrase_decapitalised', 
'n_segments_inserted', 'n_segments_removed', 'n_tokens_inserted', 'n_tokens_removed','edit_distance_token_level']
for colname in line_colnames: 
    fig = plot_epoch_line_charts(df_concat, colname)
    fig_l.append({f"pp_metrics/{colname}":fig })
    
for d in hist_config_dicts: 
    fig = plot_idx_hist(df_concat, d['colname'],d['xlabel'],d['cumulative'])
    fig_l.append({f"pp_metrics/{d['colname']}":fig })

d1 = {k: v for d in fig_l for k, v in d.items()}

run.log(d1)
wandb.finish()

## Testing and debugging 

### Verifying that the weights update each training step 

In [ ]:
#check_parameters_update(dl)  # from utils script

## Code scraps 

### Experiments around plotting average parameter updates 

In [ ]:
# def get_parameter_group_dict(): 
#     """Function to create "groups" of parameters. This is useful to check how much a group of 
#     parameters updates at an epoch. 
#     Parameter groups are hardcoded into this code for now. 
#     """
#     # Identify which parameters should be grouped together
#     isolates = ['model.shared.weight',"model.encoder.embed_positions.weight", "model.encoder.layer_norm",
#                 "model.decoder.embed_positions.weight", "model.decoder.layer_norm"]
#     layers_base = ["model.encoder.layers", "model.decoder.layers"]
#     def flatten_list(l): return list(np.concatenate(l).flat)
#     layers = flatten_list([[lyr + "." + str(o) +"." for o in list(range(16))] for lyr in layers_base])
#     parameter_groups = layers + isolates
#     # Sort the parameter groups by the order they appear in the model 
#     all_params = [name for name,_ in pp_model.named_parameters()]
#     ordering = [np.min(np.where([pg in o for o in all_params])) for pg in parameter_groups]
#     parameter_groups = [o for _,o in sorted(zip(ordering, parameter_groups))]
#     # Assign each model parameter a parameter group 
#     group_d = dict()
#     for pg in parameter_groups: 
#         name = pg[:-1] if pg in layers else pg  # remove the "." from the end of the name for the numeric layers
#         group_d[name] = [o for o in all_params if pg in o]
#     return group_d

In [ ]:
# def get_parameter_update_amount(): 
#     group_d = get_parameter_group_dict()
#     params_all_initial_d = dict(params_all_initial)
#     params_all_d = dict(params_all)
#     group_d = get_parameter_group_dict()
#     df_d = dict()
#     for k,param_l in group_d.items(): 
#         l = list()
#         for p in param_l: 
#             l.append((params_all_initial_d[p] - params_all_d[p]).abs().flatten())
#         l = torch.cat(l).cpu().detach().numpy()  # list of 1-d tensors to tensor and then to numpy
#         df_d[k] = pd.DataFrame(l).describe().values.flatten()
#     df = pd.DataFrame(df_d)
#     df.index = pd.DataFrame([1,2,3]).describe().index
#     return df 

In [ ]:
## Random code snippets

# initial_params = [(name, p.detach().clone()) for (name, p) in pp_model.named_parameters()]
# loss, reward, pp_logp = training_step(data) 
# update_d =  dict()
# for (_,old_p), (name, new_p) in zip(initial_params, pp_model.named_parameters()): 
#     update_d[name] = torch.abs(old_p - new_p).detach().flatten()     
    
#             update_d =  dict()
#             for (_,old_p), (name, new_p) in zip(initial_params, pp_model.named_parameters()): 
#                 update_d[name] = torch.abs(old_p - new_p).flatten() 
#                 print (name, torch.norm(new_p - old_p).item())  
            
#             group_d = get_parameter_group_dict()
#             initial_params_d,current_params_d = dict(initial_params),dict()
#             params_all_d = dict(params_all)
#             group_d = get_parameter_group_dict()
#             df_d = dict()
#             for k,param_l in group_d.items(): 
#                 l = list()
#                 for p in param_l: 
#                     l.append((params_all_initial_d[p] - params_all_d[p]).abs().flatten())
#                 l = torch.cat(l).cpu().detach().numpy()  # list of 1-d tensors to tensor and then to numpy
#                 df_d[k] = pd.DataFrame(l).describe().values.flatten()
#             df = pd.DataFrame(df_d)
#             df.index = pd.DataFrame([1,2,3]).describe().index

### Generating a paraphrase dataset and getting VM predictions for it

In [ ]:
# def create_paraphrase_dataset(batch, cname_input, cname_output, num_beams=32,
#                               num_return_sequences=32): 
#     """Create paraphrases for each example in the batch. Then repeat the other fields 
#         so that the resulting datase has the same length as the number of paraphrases. 
#         Key assumption is 
#         that the same number of paraphrases is created for each example.
#         batch: a dict of examples used by the `map` function from the dataset
#         cname_input: What column to create paraphrases of 
#         cname_output: What to call the column of paraphrases
#         other parameters - passed to get_paraphrases. """
    
#     # Generate paraphrases. 
#     # This can be later extended to add diversity or so on. 
#     #set_trace()
#     pp_l,probs = get_paraphrases(batch[cname_input], num_beams=num_beams,
#         num_return_sequences=num_return_sequences)
    
#     # To return paraphrases as a list of lists for batch input (not done here but might need later)
#     #     split_into_sublists = lambda l,n: [l[i:i + n] for i in range(0, len(l), n)]
#     #     pp_l = split_into_sublists(pp_l, n_seed_seqs)
#     batch[cname_output] = pp_l 
#     batch["probs"] = probs.to('cpu').numpy()
    
#     # Repeat each entry in all other columns `num_return_sequences` times so they are the same length
#     # as the paraphrase column
#     # Only works if the same number of paraphrases is generated for each phrase. 
#     # Else try something like 
#         # for o in zip(*batch.values()):
#         #     d = dict(zip(batch.keys(), o))
#         #     get_paraphrases(batch[cname_input],num_return_sequences=n_seed_seqs,num_beams=n_seed_seqs)
#         #     for k,v in d.items(): 
#         #       return_d[k] += v if k == 'text' else [v for o in range(n_paraphrases)]
#         # return return_d
#     return_d = defaultdict(list) 
#     repeat_each_item_n_times = lambda l,n: [o for o in l for i in range(n)]
#     for k in batch.keys(): 
#         if   k == cname_output: return_d[k] = batch[cname_output]
#         elif k == "probs"     : return_d[k] = batch["probs"]
#         else:                   return_d[k] = repeat_each_item_n_times(batch[k], num_return_sequences)
#     return return_d 

In [ ]:
# def get_vm_scores(ds_pp, cname_orig, cname_pp, cname_label='label', 
#                   use_metric=False, monitor=False): 
#     """Get victim model preds+probs for the paraphrase dataset.
#     """
#     assert vm_model.training == False  # checks that model is in eval mode 
#     if use_metric: 
#         metric_d = {}
#         metric_d['orig'],metric_d['pp'] = load_metric('accuracy'),load_metric('accuracy')
#     orig_probs_l,pp_probs_l = [],[]
#     if monitor: monitor = Monitor(2)  # track GPU usage and memory
    
#     def get_vm_preds(x): 
#         """Get predictions for a vector x (here a vector of documents/text). 
#         Works for a sentiment-analysis dataset (needs to be adjusted for NLI tasks)"""
#         inputs = vm_tokenizer(x, padding=True, truncation=True, return_tensors="pt")
#         inputs.to(device)
#         outputs = vm_model(**inputs, labels=labels)
#         probs = outputs.logits.softmax(1).cpu()
#         preds = probs.argmax(1)
#         return probs, preds
       
#     print("Getting victim model predictions for both original and paraphrased text.")
#     dl = DataLoader(ds_pp, batch_size=batch_size, shuffle=False, 
#                     num_workers=n_wkrs, pin_memory=True)
#     with torch.no_grad():
#         for i, data in enumerate(dl): 
#             if i % 50 == 0 : print("Now processing batch", i, "out of", len(dl))
#             labels,orig,pp = data['label'].to(device),data[cname_orig],data[cname_pp]
#             orig_probs, orig_preds = get_vm_preds(orig)            
#             pp_probs,   pp_preds   = get_vm_preds(pp)    
#             orig_probs_l.append(orig_probs); pp_probs_l.append(pp_probs)
#             if use_metric: 
#                 metric_d['orig'].add_batch(predictions=orig_preds, references=labels)
#                 metric_d['pp'].add_batch(  predictions=pp_preds,   references=labels)
#     if monitor: monitor.stop()
#     def list2tensor(l): return torch.cat(l)
#     orig_probs_t,pp_probs_t = list2tensor(orig_probs_l),list2tensor(pp_probs_l)
#     if use_metric: return orig_probs_t, pp_probs_t, metric_d
#     else:          return orig_probs_t, pp_probs_t, None

In [ ]:
# ### Generate paraphrase dataset
# num_beams = 10
# num_return_sequences = 3
# cname_input = 'text' # which text column to paraphrase
# cname_output= cname_input + '_pp'
# date = '20210825'
# fname = path_cache + '_rt_train'+ date + '_' + str(num_return_sequences)
# if os.path.exists(fname):  
#     ds_pp = datasets.load_from_disk(fname)
# else:
#     ds_pp = train.shard(200, 0, contiguous=True)
#     # Have to call with batched=True
#     # Need to set a batch size otherwise will run out of memory on the GPU card. 
#     # 64 seems to work well 
#     ds_pp = ds_pp.map(
#         lambda x: create_paraphrase_dataset(x, 
#             num_beams=num_beams, num_return_sequences=num_return_sequences,
#             cname_input=cname_input, cname_output=cname_output),
#         batched=True, batch_size=4) 
#     ds_pp.save_to_disk(fname)
#     gc.collect(); torch.cuda.empty_cache() # free up most of the GPU memory

In [ ]:
# ### Get predictions
# cname_orig = cname_input
# cname_pp = cname_output
# cname_label = 'label'
# print_metric = True
# fname = path_cache + 'results_df_'+ date + "_" + str(num_return_sequences) + ".csv"
# if os.path.exists(fname):    results_df = pd.read_csv(fname)
# else: 
#     #sim_score_t = generate_sim_scores()
#     orig_probs_t,pp_probs_t,metric_d = get_vm_scores(ds_pp, cname_orig, 
#                                                      cname_pp, cname_label,
#                                                      monitor=True, use_metric=print_metric)
#     if print_metric: 
#         print("orig vm accuracy:",       metric_d['orig'].compute())
#         print("paraphrase vm accuracy:", metric_d['pp'].compute())
#     vm_orig_scores  = torch.tensor([r[idx] for idx,r in zip(ds_pp[cname_label], orig_probs_t)])
#     vm_pp_scores    = torch.tensor([r[idx] for idx,r in zip(ds_pp[cname_label], pp_probs_t)])
#     results_df = pd.DataFrame({
#                   cname_orig: ds_pp[cname_orig],
#                   cname_pp: ds_pp[cname_pp],
#    #               'sim_score': sim_score_t,
#                   'label_true': ds_pp[cname_label], 
#                   'label_vm_orig': orig_probs_t.argmax(1),
#                   'label_vm_pp': pp_probs_t.argmax(1),
#                   'vm_orig_truelabel': vm_orig_scores,             
#                   'vm_pp_truelabel': vm_pp_scores,
#                   'vm_truelabel_change': vm_orig_scores - vm_pp_scores,
#                   'vm_orig_class0': orig_probs_t[:,0], 
#                   'vm_orig_class1': orig_probs_t[:,1], 
#                   'vm_pp_class0': pp_probs_t[:,0], 
#                   'vm_pp_class1': pp_probs_t[:,1], 
#                   })
# #    results_df['vm_truelabel_change_X_sim_score'] = results_df['vm_truelabel_change'] * results_df['sim_score']
#     results_df.to_csv(fname, index_label = 'idx')

### Testing how to keep gradients with `generate` functions

In [ ]:
# ### Testing the `generate_with_grad` function

# input_text="hello my name is Tom"
# num_return_sequences=1
# num_beams=2
# return_probs=True
# batch = pp_tokenizer(input_text, truncation=True, padding='longest', return_tensors="pt").to(device)
# generated = pp_model.generate_with_grad(**batch, return_dict_in_generate=True, output_scores=True,
#                               num_return_sequences=num_return_sequences,
#                                 num_beams=num_beams,
#                                 num_beam_groups=1,
#                                 diversity_penalty=0,
#                                 temperature=1.5, 
#                               length_penalty=1)
# print(generated)

# tgt_text = pp_tokenizer.batch_decode(generated.sequences, skip_special_tokens=True)
# print(pp_tokenizer.tokenize(tgt_text[0]))
# print(pp_tokenizer.encode(tgt_text[0]))

# # Score: score = sum_logprobs / (hyp.shape[-1] ** self.length_penalty)
# # gradient gets removed (i think) by the line 
# # beam_hyp.add(
# #   input_ids[batch_beam_idx].clone(),
# #   next_score.item())


# x=generated['scores'][5]
# print(x.max(1))
# x.max(1).values / (len(generated['scores']) ** 0.8)

In [ ]:
## An example of how to use greedy_search

# from transformers import (
# AutoTokenizer,
# AutoModelForCausalLM,
# LogitsProcessorList,
# MinLengthLogitsProcessor,
# )

# tokenizer = AutoTokenizer.from_pretrained("gpt2")
# model = AutoModelForCausalLM.from_pretrained("gpt2")

# # set pad_token_id to eos_token_id because GPT2 does not have a EOS token
# model.config.pad_token_id = model.config.eos_token_id

# input_prompt = "Today is a beautiful day, and"
# input_ids = tokenizer(input_prompt, return_tensors="pt").input_ids

# # instantiate logits processors
# logits_processor = LogitsProcessorList([
#     MinLengthLogitsProcessor(15, eos_token_id=model.config.eos_token_id),
# ])

# outputs = model.greedy_search(input_ids, logits_processor=logits_processor)

# print("Generated:", tokenizer.batch_decode(outputs, skip_special_tokens=True))

### Tensorboard setup 

In [ ]:

# from torch.utils.tensorboard import SummaryWriter
# import datetime 
# # Create writer and track to run directory 
# path_runs = './runs/'
# log_dir = path_runs + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + "/"
# writer = SummaryWriter(log_dir = log_dir)
# # stuff here logging to tensorboard
# #writer.close() # important otherwise Tensorboard eventually shuts down


### WandB artifact tables 

In [ ]:
# log predictions table to wandb, giving it a name
#  train_table_artifact = wandb.Artifact("train_samples_" + str(wandb.run.id), type="predictions")
#  valid_table_artifact = wandb.Artifact("test_samples_"   + str(wandb.run.id), type="predictions")
#train_table_artifact.add(train_table, "predictions")
#valid_table_artifact.add(valid_table, "predictions")
#wandb.run.log_artifact(train_table_artifact) 
#wandb.run.log_artifact(valid_table_artifact)

### A no_grad version of `model.generate()` adapted from transformers v4.5.0

In [ ]:
# from typing import Any, Callable, Dict, Iterable, List, Mapping, Optional, Union
# from transformers.generation_beam_search import BeamScorer, BeamSearchScorer
# import torch


# def generate_with_grad(
#     self,
#     input_ids: Optional[torch.LongTensor] = None,
#     max_length: Optional[int] = None,
#     min_length: Optional[int] = None,
#     do_sample: Optional[bool] = None,
#     early_stopping: Optional[bool] = None,
#     num_beams: Optional[int] = None,
#     temperature: Optional[float] = None,
#     top_k: Optional[int] = None,
#     top_p: Optional[float] = None,
#     repetition_penalty: Optional[float] = None,
#     bad_words_ids: Optional[Iterable[int]] = None,
#     bos_token_id: Optional[int] = None,
#     pad_token_id: Optional[int] = None,
#     eos_token_id: Optional[int] = None,
#     length_penalty: Optional[float] = None,
#     no_repeat_ngram_size: Optional[int] = None,
#     encoder_no_repeat_ngram_size: Optional[int] = None,
#     num_return_sequences: Optional[int] = None,
#     max_time: Optional[float] = None,
#     decoder_start_token_id: Optional[int] = None,
#     use_cache: Optional[bool] = None,
#     num_beam_groups: Optional[int] = None,
#     diversity_penalty: Optional[float] = None,
#     prefix_allowed_tokens_fn: Optional[Callable[[int, torch.Tensor], List[int]]] = None,
#     output_attentions: Optional[bool] = None,
#     output_hidden_states: Optional[bool] = None,
#     output_scores: Optional[bool] = None,
#     return_dict_in_generate: Optional[bool] = None,
#     forced_bos_token_id: Optional[int] = None,
#     forced_eos_token_id: Optional[int] = None,
#     remove_invalid_values: Optional[bool] = None,
#     **model_kwargs):
#     # set init values
#     num_beams = num_beams if num_beams is not None else self.config.num_beams
#     num_beam_groups = num_beam_groups if num_beam_groups is not None else self.config.num_beam_groups
#     max_length = max_length if max_length is not None else self.config.max_length
#     do_sample = do_sample if do_sample is not None else self.config.do_sample
#     num_return_sequences = (
#         num_return_sequences if num_return_sequences is not None else self.config.num_return_sequences
#     )

#     pad_token_id = pad_token_id if pad_token_id is not None else self.config.pad_token_id
#     bos_token_id = bos_token_id if bos_token_id is not None else self.config.bos_token_id
#     eos_token_id = eos_token_id if eos_token_id is not None else self.config.eos_token_id

#     output_scores = output_scores if output_scores is not None else self.config.output_scores
#     output_attentions = output_attentions if output_attentions is not None else self.config.output_attentions
#     output_hidden_states = (
#         output_hidden_states if output_hidden_states is not None else self.config.output_hidden_states
#     )
#     return_dict_in_generate = (
#         return_dict_in_generate if return_dict_in_generate is not None else self.config.return_dict_in_generate
#     )

#     model_kwargs["output_attentions"] = output_attentions
#     model_kwargs["output_hidden_states"] = output_hidden_states

#     if input_ids is None:
#         # init `input_ids` with bos_token_id
#         input_ids = self._prepare_input_ids_for_generation(bos_token_id, model_kwargs.get("encoder_outputs"))

#     if model_kwargs.get("attention_mask", None) is None:
#         # init `attention_mask` depending on `pad_token_id`
#         model_kwargs["attention_mask"] = self._prepare_attention_mask_for_generation(
#             input_ids, pad_token_id, eos_token_id
#         )

#     # special case if pad_token_id is not defined
#     if pad_token_id is None and eos_token_id is not None:
#         logger.warning(f"Setting `pad_token_id` to `eos_token_id`:{eos_token_id} for open-end generation.")
#         pad_token_id = eos_token_id

#     # Storing encoder_input_ids for logits_processor that could use them
#     encoder_input_ids = input_ids if self.config.is_encoder_decoder else None

#     if self.config.is_encoder_decoder:
#         # add encoder_outputs to model_kwargs
#         model_kwargs = self._prepare_encoder_decoder_kwargs_for_generation(input_ids, model_kwargs)

#         # set input_ids as decoder_input_ids
#         if "decoder_input_ids" in model_kwargs:
#             input_ids = model_kwargs.pop("decoder_input_ids")
#         else:
#             input_ids = self._prepare_decoder_input_ids_for_generation(
#                 input_ids, decoder_start_token_id=decoder_start_token_id, bos_token_id=bos_token_id
#             )

# #         if "encoder_outputs" not in model_kwargs or not isinstance(model_kwargs["encoder_outputs"], ModelOutput):
# #             raise ValueError("Make sure that `model_kwargs` include `encoder_outputs` of type `ModelOutput`.")
#     if input_ids.shape[-1] >= max_length:
#         input_ids_string = "decoder_input_ids" if self.config.is_encoder_decoder else "input_ids"
#         logger.warning(
#             f"Input length of {input_ids_string} is {input_ids.shape[-1]}, but ``max_length`` is set to {max_length}."
#             "This can lead to unexpected behavior. You should consider increasing ``config.max_length`` or ``max_length``."
#         )

#     # determine generation mode
#     is_greedy_gen_mode = (num_beams == 1) and (num_beam_groups == 1) and do_sample is False
#     is_sample_gen_mode = (num_beams == 1) and (num_beam_groups == 1) and do_sample is True
#     is_beam_gen_mode = (num_beams > 1) and (num_beam_groups == 1) and do_sample is False
#     is_beam_sample_gen_mode = (num_beams > 1) and (num_beam_groups == 1) and do_sample is True
#     is_group_beam_gen_mode = (num_beams > 1) and (num_beam_groups > 1)
#     if num_beam_groups > num_beams:
#         raise ValueError("`num_beam_groups` has to be smaller or equal to `num_beams`")
#     if is_group_beam_gen_mode and do_sample is True:
#         raise ValueError(
#             "Diverse beam search cannot be used in sampling mode. Make sure that `do_sample` is set to `False`."
#         )

#     # set model_kwargs
#     model_kwargs["use_cache"] = use_cache

#     # get distribution pre_processing samplers
#     logits_processor = self._get_logits_processor(
#         repetition_penalty=repetition_penalty,
#         no_repeat_ngram_size=no_repeat_ngram_size,
#         encoder_no_repeat_ngram_size=encoder_no_repeat_ngram_size,
#         encoder_input_ids=encoder_input_ids,
#         bad_words_ids=bad_words_ids,
#         min_length=min_length,
#         max_length=max_length,
#         eos_token_id=eos_token_id,
#         forced_bos_token_id=forced_bos_token_id,
#         forced_eos_token_id=forced_eos_token_id,
#         prefix_allowed_tokens_fn=prefix_allowed_tokens_fn,
#         num_beams=num_beams,
#         num_beam_groups=num_beam_groups,
#         diversity_penalty=diversity_penalty,
#         remove_invalid_values=remove_invalid_values,
#     )

#     stopping_criteria = self._get_stopping_criteria(
#         max_length=max_length,
#         max_time=max_time,
#     )

#     if is_greedy_gen_mode:
#         if num_return_sequences > 1:
#             raise ValueError(
#                 f"num_return_sequences has to be 1, but is {num_return_sequences} when doing greedy search."
#             )

#         # greedy search
#         return self.greedy_search(
#             input_ids,
#             logits_processor=logits_processor,
#             stopping_criteria=stopping_criteria,
#             max_length=max_length,
#             pad_token_id=pad_token_id,
#             eos_token_id=eos_token_id,
#             output_scores=output_scores,
#             return_dict_in_generate=return_dict_in_generate,
#             **model_kwargs,
#         )

#     elif is_sample_gen_mode:
#         # get probability distribution warper
#         logits_warper = self._get_logits_warper(
#             top_k=top_k, top_p=top_p, temperature=temperature, num_beams=num_beams
#         )

#         # expand input_ids with `num_return_sequences` additional sequences per batch
#         input_ids, model_kwargs = self._expand_inputs_for_generation(
#             input_ids,
#             expand_size=num_return_sequences,
#             is_encoder_decoder=self.config.is_encoder_decoder,
#             **model_kwargs,
#         )

#         # sample
#         return self.sample(
#             input_ids,
#             logits_processor=logits_processor,
#             logits_warper=logits_warper,
#             stopping_criteria=stopping_criteria,
#             max_length=max_length,
#             pad_token_id=pad_token_id,
#             eos_token_id=eos_token_id,
#             output_scores=output_scores,
#             return_dict_in_generate=return_dict_in_generate,
#             **model_kwargs,
#         )

#     elif is_beam_gen_mode:
#         batch_size = input_ids.shape[0]

#         length_penalty = length_penalty if length_penalty is not None else self.config.length_penalty
#         early_stopping = early_stopping if early_stopping is not None else self.config.early_stopping

#         if num_return_sequences > num_beams:
#             raise ValueError("`num_return_sequences` has to be smaller or equal to `num_beams`.")

#         beam_scorer = BeamSearchScorer(
#             batch_size=batch_size,
#             max_length=max_length,
#             num_beams=num_beams,
#             device=self.device,
#             length_penalty=length_penalty,
#             do_early_stopping=early_stopping,
#             num_beam_hyps_to_keep=num_return_sequences,
#         )
#         # interleave with `num_beams`
#         input_ids, model_kwargs = self._expand_inputs_for_generation(
#             input_ids, expand_size=num_beams, is_encoder_decoder=self.config.is_encoder_decoder, **model_kwargs
#         )
#         #with torchsnooper.snoop(depth=4, max_variable_length=200, normalize=True):
#         return self.beam_search(
#             input_ids,
#             beam_scorer,
#             logits_processor=logits_processor,
#             stopping_criteria=stopping_criteria,
#             max_length=max_length,
#             pad_token_id=pad_token_id,
#             eos_token_id=eos_token_id,
#             output_scores=output_scores,
#             return_dict_in_generate=return_dict_in_generate,
#             **model_kwargs,
#         )

#     elif is_beam_sample_gen_mode:
#         logits_warper = self._get_logits_warper(
#             top_k=top_k, top_p=top_p, temperature=temperature, num_beams=num_beams
#         )

#         batch_size = input_ids.shape[0] * num_return_sequences

#         length_penalty = length_penalty if length_penalty is not None else self.config.length_penalty
#         beam_scorer = BeamSearchScorer(
#             batch_size=batch_size,
#             max_length=max_length,
#             num_beams=num_beams,
#             device=self.device,
#             length_penalty=length_penalty,
#             do_early_stopping=early_stopping,
#         )

#         # interleave with `num_beams * num_return_sequences`
#         input_ids, model_kwargs = self._expand_inputs_for_generation(
#             input_ids,
#             expand_size=num_beams * num_return_sequences,
#             is_encoder_decoder=self.config.is_encoder_decoder,
#             **model_kwargs,
#         )

#         return self.beam_sample(
#             input_ids,
#             beam_scorer,
#             logits_processor=logits_processor,
#             logits_warper=logits_warper,
#             stopping_criteria=stopping_criteria,
#             max_length=max_length,
#             pad_token_id=pad_token_id,
#             eos_token_id=eos_token_id,
#             output_scores=output_scores,
#             return_dict_in_generate=return_dict_in_generate,
#             **model_kwargs,
#         )

#     elif is_group_beam_gen_mode:
#         batch_size = input_ids.shape[0]

#         length_penalty = length_penalty if length_penalty is not None else self.config.length_penalty
#         early_stopping = early_stopping if early_stopping is not None else self.config.early_stopping

#         if num_return_sequences > num_beams:
#             raise ValueError("`num_return_sequences` has to be smaller or equal to `num_beams`.")

#         if num_beams % num_beam_groups != 0:
#             raise ValueError("`num_beams` should be divisible by `num_beam_groups` for group beam search.")

#         diverse_beam_scorer = BeamSearchScorer(
#             batch_size=batch_size,
#             max_length=max_length,
#             num_beams=num_beams,
#             device=self.device,
#             length_penalty=length_penalty,
#             do_early_stopping=early_stopping,
#             num_beam_hyps_to_keep=num_return_sequences,
#             num_beam_groups=num_beam_groups,
#         )
#         # interleave with `num_beams`
#         input_ids, model_kwargs = self._expand_inputs_for_generation(
#             input_ids, expand_size=num_beams, is_encoder_decoder=self.config.is_encoder_decoder, **model_kwargs
#         )
#         return self.group_beam_search(
#             input_ids,
#             diverse_beam_scorer,
#             logits_processor=logits_processor,
#             stopping_criteria=stopping_criteria,
#             max_length=max_length,
#             pad_token_id=pad_token_id,
#             eos_token_id=eos_token_id,
#             output_scores=output_scores,
#             return_dict_in_generate=return_dict_in_generate,
#             **model_kwargs,
#         )


### Old eval/wandb functions 

In [ ]:
# def print_eval_metrics(preds, split):
#     ### Might be obselete ###
#     print(f"{split} paraphrases:", preds['pp_l'])
#     print(f"{split} VM scores:",    np.round(preds['vm_score'],3))
#     print(f"{split} ROUGE scores:", np.round(preds['rouge_score'],3))
#     if normalise_rewards: print(f"{split} unnormalised rewards:", preds['orig_reward'])
#     print(f"{split} rewards:", round_t(preds['reward'], 3))
#     print(f"{split} avg reward:", torch.mean(preds['reward']).item())
#     print(f"{split} logp:", round_t(preds['pp_logp'], 3))
#     print(f"{split} avg logp:", torch.mean(preds['pp_logp']).item())
#     print(f"{split} loss:", train_set_preds['loss'].item())

In [ ]:
# def log_wandb_metrics(results_d, split): 
#     """flattens lists of metrics to wandb acceptable form. obselete now but might be useful later"""
#     #### MIGHT BE OBSELETE #####
#     # Log numeric data 
#     # Convert all lists of values to wandb format. Scalars are unchanged
#     d = dict()
#     orig_keys = results_d.keys()
#     for k,v in results_d.items(): 
#         if type(v) is list: 
#             if type(v[0]) == int or type(v[0]) == float:  # we handle strings differently
#                 d1 = {f"eval/{split}/examples/{k}/{i}": o for i,o in enumerate(v)}  # list -> dict of len(v) scalars
#                 d = {**d, **d1}  # merge dicts
#         else: 
#             d[f"{split}/{k}"] = v
#     d['epoch'] = epoch
#     wandb.log(d) 

In [ ]:
### An attempt to rename wandb columns of a table. This seemed to break something internally inside the table.
#
#     def rename_wandb_column(table, old, new): 
#         if old not in table.columns: 
#             warnings.warn(f"{old} not in columns of table. Skipping. Columns of table are {table.columns}")
#         else: 
#             idx = [i for i,o in enumerate(table.columns) if o == old][0]
#             table.columns[idx] = new
        
    # Can't just originally name these 'orig' and 'pp' because they don't match key names used in `eval_dl`
    # NOTE: this seems to break the internals of wandb. might have to just live with the names. 
#     rename_wandb_column(train_table, old='orig_l', new='orig')
#     rename_wandb_column(train_table, old='pp_l',   new='pp')
#     rename_wandb_column(valid_table,  old='orig_l', new='orig')
#     rename_wandb_column(valid_table,  old='pp_l',   new='pp')